###1. ngrok 가입
- https://dashboard.ngrok.com/signup
- Getting Started -> Yout authtoken

In [ ]:
!pip install flask pyngrok

In [ ]:
!ngrok config add-authtoken 261FpEwUH0gqBvM60vQWoJTop61_6Tc3cbQaS2WnRdFAS6GeM

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


#2. fastapi 설치
- fastapi :  REST API 서버 개발, 머신러닝 모델 서빙 API, 실시간 데이터 처리 API
- uvicorn :  FastAPI를 실행시키는 ASGI 서버(웹 서버, Asynchronous Server Gateway Interface, WSGI(Web Server Gateway Interface)의 비동기 버전으로 Python 웹 서버(예: uvicorn)와 웹 프레임워크(예: FastAPI) 사이의 표준 인터페이스)
- nest-asyncio : Jupyter/Colab 같은 이미 asyncio(비동기 I/O 작업을 처리하기 위한 기반 프레임워크) 루프가 돌아가는 환경에서
추가로 asyncio 서버(FastAPI 등)를 중복으로 실행할 수 있도록 도와주는 패치 도구,
Colab, Jupyter, Spyder 등에서 uvicorn.run() 직접 실행 가능하게 해줌

In [ ]:
!pip install fastapi uvicorn nest-asyncio

In [ ]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import threading

In [ ]:
# FastAPI 앱 정의
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello from FastAPI + ngrok!"}

@app.get("/hello")
def say_hello(name: str = "world"):
    return {"greeting": f"Hello, {name}!"}

# asyncio 루프 재설정
nest_asyncio.apply()

# FastAPI 서버 실행 함수
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# 서버를 백그라운드에서 실행
threading.Thread(target=run).start()

# ngrok으로 8000 포트 열기
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


INFO:     Started server process [455]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://a9c74e4a8940.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
public_url.public_url

'https://a9c74e4a8940.ngrok-free.app'

- pydantic

In [7]:
from pydantic import BaseModel

class User(BaseModel):    #BaseModel 상속
    name: str
    age: int

data = {"name": "Alice", "age": "25"}  # 문자열 "25"도 int로 변환됨
user = User(**data) #User(name='Alice', age='25')와 같은 효과. data 딕셔너리의 키-값 쌍을 User 클래스의 생성자에 키워드 인수(keyword arguments) 로 전달
print(user)


name='Alice' age=25


In [12]:
user.model_dump() #user.dict()
user.model_dump_json()

'{"name":"Alice","age":25}'

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@app.post("/predict")
def predict(data: IrisInput):   #data는 IrisInput이어야 함, 타잎 힌트
    return {"received": data.dict()}

In [ ]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=2)      #Field()는 필드에 대해 기본값 설정 + 제약조건(validation rule) 을 명시, ...는 이 필드가 필수(required)임 의미
    price: float = Field(..., gt=0)

- pydantic의 Field 옵션
 - default:	기본값 설정 (또는 ...로 필수 지정), 예:	Field(10) or Field(...)
 - title:	필드 제목 (문서화에 사용됨), 예:		Field(..., title="Product Name")
 - description:	설명 (API 문서 등에 활용됨), 예:		Field(..., description="제품 이름")
 - min_length:	최소 문자열 길이, 예:		Field(..., min_length=3)
 - max_length:	최대 문자열 길이, 예:		Field(..., max_length=20)
 - gt	(greater than): 이 값보다 커야 함, 예:		Field(..., gt=0)
 - ge	(greater or equal): 이상, 예:		Field(..., ge=0)
 - lt	(less than): 미만, 예:		Field(..., lt=100)
 - le	(less or equal): 이하, 예:		Field(..., le=100)
 - regex:	문자열 정규표현식 검증, 예:		Field(..., regex="^item_[0-9]+$")
 - example:	OpenAPI 문서에서 예시로 보여줄 값, 예:		Field(..., example="TV")
 - alias:	JSON 키와 다른 필드명을 사용할 때, 예:		Field(..., alias="product_name")
 - deprecated:	더 이상 사용되지 않음을 명시, 예:	 (OpenAPI 문서에서 표시됨)	Field(..., deprecated=True)

In [ ]:
# 유효한 입력 예:
{"name": "TV", "price": 999.99}

# 유효하지 않은 입력 예:
{"name": "", "price": -3}

{'name': '', 'price': -3}

In [ ]:
{
  "detail": [
    {
      "loc": ["body", "name"],
      "msg": "ensure this value has at least 2 characters",
      "type": "value_error.any_str.min_length",
    },
    {
      "loc": ["body", "price"],
      "msg": "ensure this value is greater than 0",
      "type": "value_error.number.not_gt"
    }
  ]
}


{'detail': [{'loc': ['body', 'name'],
   'msg': 'ensure this value has at least 2 characters',
   'type': 'value_error.any_str.min_length'},
  {'loc': ['body', 'price'],
   'msg': 'ensure this value is greater than 0',
   'type': 'value_error.number.not_gt'}]}

In [ ]:
user = User(name="Bob", age=40)
print(user.dict())   #  {'name': 'Bob', 'age': 40}  #model_dump()
print(user.json())   # '{"name": "Bob", "age": 40}' #model_dump_json()

{'name': 'Bob', 'age': 40}
{"name":"Bob","age":40}


/tmp/ipython-input-1380281954.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.dict())   #  {'name': 'Bob', 'age': 40}  #model_dump()
/tmp/ipython-input-1380281954.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(user.json())   # '{"name": "Bob", "age": 40}' #model_dump_json()


#3. fastapi 응용
- 런타임 재시작 후 실행

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from pyngrok import ngrok
from typing import List
import uvicorn
import nest_asyncio
import threading

# -------------------
# 1. 모델 학습
# -------------------
iris = load_iris()
X, y = iris.data, iris.target

model = DecisionTreeClassifier()
model.fit(X, y)

# -------------------
# 2. FastAPI 정의
# -------------------
app = FastAPI()

class ListInput(BaseModel):
    values: List[float]

class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@app.post("/predict")
def predict(data: IrisInput):
    input_data = [[
        data.sepal_length,
        data.sepal_width,
        data.petal_length,
        data.petal_width
    ]]
    prediction = model.predict(input_data)[0]
    label = iris.target_names[prediction]
    return {"prediction": int(prediction), "label": label}

@app.post("/predict2")
def predict(data: ListInput):
    prediction = model.predict([data.values])[0]
    label = iris.target_names[prediction]
    return {"prediction": int(prediction), "label": label}

# -------------------
# 3. 서버 실행 및 ngrok 연결
# -------------------
nest_asyncio.apply()
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)
threading.Thread(target=run).start()
public_url = ngrok.connect(8000)
print(f"Public FastAPI URL: {public_url}")


INFO:     Started server process [5502]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public FastAPI URL: NgrokTunnel: "https://7fe4b686bd40.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
public_url.public_url+"/docs"  #swagger ui

'https://7fe4b686bd40.ngrok-free.app/docs'

- api 사용

In [ ]:
import requests

url = public_url.public_url+"/predict"
X_new = {
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
}

response = requests.post(url, json=X_new)
print(response.json())

INFO:     34.74.1.41:0 - "POST /predict HTTP/1.1" 200 OK
{'prediction': 0, 'label': 'setosa'}


In [ ]:
import requests

url = public_url.public_url+"/predict2"
X_new = {"values": [5.1, 3.5, 1.4, 0.2]}
response = requests.post(url, json=X_new)
print(response.json())

INFO:     34.74.1.41:0 - "POST /predict2 HTTP/1.1" 200 OK
{'prediction': 0, 'label': 'setosa'}
